In [3]:
import numpy as np
import pandas as pd
import xlsxwriter
from scipy import stats
import math
import yfinance as yf

To begin, we are going to do a simple model with only 1 value metric - price to earnings ratio (take their stock price and divide by how much earnings you expect them to have over the next year.)

In [5]:
# price to earnings ratio = price per share / earnings per share
# earnings per share = net income / end of period common shares outstanding
# common shares = shares you normally trade
# outstanding shares = shares currently owned by all shareholders (excluding treasury shares, which are owned by the company itself)
# pe ratio can have different calculations.
aapl = yf.Ticker("AAPL")
price = aapl.info['previousClose']
net_income = aapl.info['netIncomeToCommon']
outstanding_shares = aapl.info['sharesOutstanding']
eps = net_income / outstanding_shares
pe_ratio = price / eps
pe_ratio

33.335463728191